# Beet segmentation model application

Date: 18.02.2024  
Authors: Gustav Schimmer & Philipp Friedrich

**This notebook is purposed for explaining the application of the previously trained YOLOv6 algorithm in detecting sugar beet plants on georeferenzed images.**  

## Import necessary libraries

In [1]:
from ultralytics import YOLO
import os
import geopandas as gpd
from shapely.geometry import Polygon
from osgeo import gdal
import numpy as np

C:\Users\phili\AppData\Local\Temp\ipykernel_156604\2977665569.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


## Load previously trained YOLOv6 model using ultralytics package and provide some test image

In [2]:
model = YOLO("../../YOLOv6/runs/detect/train/weights/best.pt")  # pretrained YOLOv6n model

Now we can load a test image...

In [3]:
image_file = "../../data/20230514/field_1_orthophoto_tiles/tile_1536_11264.tif"

# tile_1536_11264
# tile_1024_10240
# try these two files

## Apply model to image and extract bounding box coordinates

This line will apply the model to our provided image.

In [7]:
# Apply model to image
results = model(image_file)
type(results)


image 1/1 C:\Users\phili\Documents\Studium\Master_Geographie\5_Semester\S_Deep_Learning\beet-segmentation\src\jupyter_notebooks\..\..\data\20230514\field_1_orthophoto_tiles\tile_1536_11264.tif: 512x512 2 sugar beets, 53.1ms
Speed: 2.1ms preprocess, 53.1ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)


list

After model application, we can extract the bounding box coordinates. Note that the coordinates are pixel coordinates! They need to be transformed to geographical coordinates later.

In [8]:
for result in results:
    print(type(result))
    boxes = result.boxes
    print(type(boxes))
    if boxes.shape[0] != 0:
        pixel_xmin = boxes.xyxy[0][0].item()
        pixel_ymin = boxes.xyxy[0][1].item()
        pixel_xmax = boxes.xyxy[0][2].item()
        pixel_ymax = boxes.xyxy[0][3].item()
    else:
        continue

<class 'ultralytics.engine.results.Results'>
<class 'ultralytics.engine.results.Boxes'>


We can now print the pixel coordinates and see whats there. The bounding box coordinates are stored as xmin, ymin, xmax, ymax. 

In [6]:
print(pixel_xmin, pixel_ymin, pixel_xmax, pixel_ymax)

243.48814392089844 149.0165252685547 375.0606689453125 246.4772491455078


## Convert pixel coordinates to geographic coordinates

To convert the pixel coordinates to geographic coordinates we first write a function that does the job:

In [7]:
# This function will convert pixel coordinates to geographic coordinates
def pixel_to_geo(pixel_x, pixel_y, geotransform):
    geo_x = geotransform[0] + pixel_x * geotransform[1] + pixel_y * geotransform[2]
    geo_y = geotransform[3] + pixel_x * geotransform[4] + pixel_y * geotransform[5]
    return geo_x, geo_y

#### Get transformation information

Before we can convert the coordinates we need to get the transformation information from the georeferenced image! Therefore we need to load the image using a geospatial raster library like GDAL. Afterwards we extract the transformation information.

In [8]:
# Load the image using GDAL
dataset = gdal.Open(image_file)
if dataset is None:
    raise FileNotFoundError("Image file not found")

# Get geotransform information
geotransform = dataset.GetGeoTransform()

C:\Users\phili\anaconda3\envs\advgeo\lib\site-packages\osgeo\gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Let's see what's in there:

In [9]:
print(geotransform)

(576496.6226073955, 0.00040246724450860487, 0.0, 5516317.77482754, 0.0, -0.00040246724450860487)


#### Coordinate conversion

No we apply the function written above to transform the bounding box coordinates from pixel to geographic coordinates.

In [10]:
# Convert pixel coordinates to geographic coordinates
geo_xmin, geo_ymin = pixel_to_geo(pixel_xmin, pixel_ymin, geotransform)
geo_xmax, geo_ymax = pixel_to_geo(pixel_xmax, pixel_ymax, geotransform)
print("Geographic coordinates:", geo_xmin, geo_ymin, geo_xmax, geo_ymax)

# Close the dataset
dataset = None

Geographic coordinates: 576496.7206033979 5516317.71485327 576496.7735570294 5516317.675628521


## Create polygon and export bounding box

#### Create polygon

Now that we have geographic coordinates we can use Shapely to reconstruct the bounding box as polygon.

In [11]:
# Create a bounding box as a Shapely Polygon object
bounding_box = Polygon([(geo_xmin, geo_ymin), (geo_xmax, geo_ymin), (geo_xmax, geo_ymax), (geo_xmin, geo_ymax)])

# Print the bounding box
print("Bounding box as a Shapely Polygon object:", bounding_box)

Bounding box as a Shapely Polygon object: POLYGON ((576496.7206033979 5516317.71485327, 576496.7735570294 5516317.71485327, 576496.7735570294 5516317.675628521, 576496.7206033979 5516317.675628521, 576496.7206033979 5516317.71485327))


#### Export bounding box

For propper export we create a geopandas dataframe. After defining the output file we can export the dataframe containing our bounding box. As driver we use GeoJSON. Our bounding box is now ready to be used for further analysis, for example in a GIS Software.

In [12]:
# Create a GeoDataFrame with the bounding box
gdf = gpd.GeoDataFrame(geometry=[bounding_box])

# Define the output file path
output_file = "bounding_box.geojson"

# Export the GeoDataFrame to a GeoJSON file
gdf.to_file(output_file, driver='GeoJSON')

print("Bounding box exported to:", output_file)

Bounding box exported to: bounding_box.geojson
